![Xilinx Logo](images/xilinx_logo.png "Xilinx Logo")

# 1. Introduction

This notebook demonstrates how to configure a V4L2 device, launches endpoint application which processes media content through filter(optional) and transfers processed content to host via PCIe.  

Depending on the usecase following Gstreamer elements are used:
* The ``pciesrc`` element is used to receive video frames from host machine.
* The ``sdxfilter2d`` element is used to implement a 2D convolution filter.
* The ``pciesink`` element is used to send video frames to host mahine in zero copy fashion.

Three different use cases are demonstrated using gstreamer based pcie Endpoint application.
* Capture from v4l2 device (MIPI), process the frames using filter2d plugin and display on host machine using pciesink.

* Receive a video file from host machine using pciesrc, process the received frames using filter2d plugin and send processed video frames to host machine through pciesink.

* Receive a video file from host machine using pciesrc and send video frames to host machine through pciesink without processing.


Based on the control information from the host application endpoint application will execute anyone of the above mentioned usecases.

All above usecases runs 30 fps and suppots below resolutions :

`` 1920x1080 ``
`` 3840x2160 ``

In [ ]:
res = "2160p" # Change the resolution, string to 1080p or 2160p (mipi only), resolution should be same as the one selected on host application. 
if res == "1080p": 
    !xmediactl_1080.sh
else :
    !xmediactl.sh

Set XRT related environment variables.

In [ ]:
%env XILINX_XRT = /usr
%env XCL_BINDIR = /media/sd-mmcblk0p1

In [ ]:
!pcie_gst_app #for fps related debugs go through Readme.txt in the pcie-gst-app recipe

<center>Copyright© 2021 Xilinx</center>